In [11]:
import pandas as pd
import re
import PyPDF2
from PyPDF2 import PdfReader
import pypdf
import docx2txt
import string

In [12]:
generals4041 = pd.read_csv("generals40-41.csv")
repressions_main_table = pd.read_csv("Repressions.csv")
generals45 = pd.read_excel("generals40-45.xls")
generals4041 = generals4041.fillna('')
generals4041["name_title"] = generals4041["Фамилия"] + " " + generals4041["Имя"] + " " + generals4041["Отчество"] + " " + generals4041["Воинское звание"]
generals4041["name"] = generals4041["Фамилия"] + " " + generals4041["Имя"] + " " + generals4041["Отчество"]
list_names_titles_4041 = list(generals4041["name_title"])
list_names_4041 = list(generals4041["name"])
list_names_4041 = [value for value in list_names_4041 if value != '  ']
generals4041.head(5)

,Фамилия,Имя,Отчество,Воинское звание,Дата,Номер,name_title,name
0,АБАНЬКИН,Павел,Сергеевич,Контр-адмирал,5/21/41,1394.0,АБАНЬКИН Павел Сергеевич Контр-адмирал,АБАНЬКИН Павел Сергеевич
1,АБРАМИДЗЕ,Павел,Ивлианович,Генерал-майор,6/4/40,945.0,АБРАМИДЗЕ Павел Ивлианович Генерал-майор,АБРАМИДЗЕ Павел Ивлианович
2,АБРАМОВ,Николай,Осипович,Контр-адмирал,6/4/40,946.0,АБРАМОВ Николай Осипович Контр-адмирал,АБРАМОВ Николай Осипович
3,АБСАЛЯМОВ,Манзакир,,Генерал-майор,6/4/40,945.0,АБСАЛЯМОВ Манзакир Генерал-майор,АБСАЛЯМОВ Манзакир
4,АВВАКУМОВ,Яков,Александрович,Генерал-майор,6/4/40,945.0,АВВАКУМОВ Яков Александрович Генерал-майор,АВВАКУМОВ Яков Александрович


In [13]:
generals4041["lurie"] = ""
generals4041["komcory"] = ""
generals4041["komandarmy"] = ""
generals4041["komdivy"] = ""
generals4041["tarkhova"] = ""
generals4041["other_source"] = ""

In [14]:
list_names_4041 = [value for value in list_names_4041 if value != '  ']
list_names_4041[:10]

['АБАНЬКИН Павел Сергеевич',
 'АБРАМИДЗЕ Павел Ивлианович',
 'АБРАМОВ Николай Осипович',
 'АБСАЛЯМОВ Манзакир ',
 'АВВАКУМОВ Яков Александрович',
 'АВЕРКИН Дмитрий Иванович',
 'АДАМОВИЧ Владимир Александрович',
 'АДРИАШЕНКО Виктор Иванович',
 'АКИМОВ Степан Дмитриевич',
 'АКУЛИН Михаил Иванович']

In [15]:
len(list_names_4041)

1280

In [16]:
#sample = df.sample(n=50)
#sample["name"] = sample["Фамилия"] + " " + sample["Имя"] + " " + sample["Отчество"] + " " + sample["Воинское звание"]
#sample.to_csv("generals40_sample.csv")

# Read PDFs

In [89]:
komcory = docx2txt.process("comcory.docx")
komcory2 = docx2txt.process("comcory2.docx")
komandarmy = PyPDF2.PdfReader('comandarmy.pdf')
lurie = PyPDF2.PdfReader('lurie2.pdf')
komdivy_p1 = PyPDF2.PdfReader('Комдивы - Соловьев - ч.1 - Командиры дивизий Красной Армии 1941-1945 гг.pdf')
komdivy_p2 = PyPDF2.PdfReader('Комдивы - Соловьев - ч.2 - Командиры дивизий Красной Армии 1941-1945 гг.pdf')
komdivy_p3 = PyPDF2.PdfReader('Комдивы - Соловьев - ч.3 - Командиры дивизий Красной Армии 1941-1945 гг.pdf')

# Лурье

In [18]:
lurie_text = ""
for page in lurie.pages:
    lurie_text = lurie_text + page.extract_text()
lurie_text = lurie_text.replace("-\n", "")
lurie_text = lurie_text.replace("- \n", "")


delete_dict = {sp_character: '' for sp_character in string.punctuation}
table = str.maketrans(delete_dict)


lurie_text1 = lurie_text.split("СОДЕРЖАНИЕ")[1]
lurie_text1 = [name.translate(table).split(" ")[0] + " " + name.translate(table).split(" ")[1] + " " + name.translate(table).split(" ")[2] for name in lurie_text1.split("\n") if len(name.translate(table).split(" ")) > 2]
lurie_text1 = lurie_text1[1:375]
lurie_list_names = lurie_text1
print(f"Total numder of Lurie names: {len(lurie_list_names)}")


counter = 0
lurie_matches = []
for name in list_names_4041:
    if name in lurie_text:
        #print(name)
        counter +=1
        lurie_matches.append(name)

print(f"Matches in Lurie: {counter}")

Total numder of Lurie names: 374
Matches in Lurie: 120


In [ ]:
lurie_text

In [ ]:
# Составление словаря, где ключи - имена, а значения - соответствующие описания
data = {}
pattern = re.compile(r'\b[А-ЯЁ][а-яё]+\s+[А-ЯЁ][а-яё]+\s+[А-ЯЁ][а-яё]+')
pattern_matches = pattern.findall(lurie_text)

list_to_drop = ['Герой\nСоветского\nСоюза', 'ГеройСоветскогоСоюза', 'Постановлением Совета Министров', 'Трудового Красного Знамени', 
               'Республика Марий Эл', 'Депутат Верховного Совета', 'Литл Адмирал Флота', 'Советского Союза Исаков',
               'Адмирал Флота Советского', 'За Советскую Карелию', 'Указе Президиума Верховного', 'Указом Президиума Верховного',
               'Тыл Советских Вооруженных', 'Ставке Маршала Советского', 'Императора Петра Великого', 'Депутат Верховного Совета',
               'Народная Республика Болгария', 'Венгерская Народная Республика', 'Звезда Румынской Народной', 'Адмирал Иван Грен',
               'Герои Социалистического Труда', 'Авиация Российского Флота', 'Советских Вооруженных Сил', 'Императора Петра Великого',
               'Член Военного Совета', 'Адмирал Флота Сов', 'Член Президиума Сов', 'Союза Исаков Иван', 'Боевого Красного Знамени', 
               'Объединенной Арабской Республике', 'Герои Советского Союза', 'Авиация Краснознаменного Черноморского', 
                'Китайской Народной Республики', 'Депутат Верховного Совета', 'Адмирал Флота Советского', 'Героях Советского Союза',
               'Авиация Краснознаменного Черноморского', 'Через Северный Ледовитый', 'Ренными Силами Республики', 'Части Красной Армии',
               'Союза Иван Степанович', 'Герой Социалистического Труда', 'Герой Арктики Иван', 'Памяти Петра Федоровича', 
                'Герои Советского Союза', 'Адмирал Иван Иванович', 'Адмирал Флота Советского', 'Союза Иван Степанович',
                'Маршала Советского Союза', 'Авиация Краснозкаменного Червоморского', 'Чеки Окончил Тифлисский',
                'Постановлением Совета Министров', 'Литл Краснознаменный Балтийский', 'Вестник Архива Армении', 'Командуя Новороссийской Военно',
                'Звезда Румынской Народной', 'Героя Советского Союза', 'Трудового Краеного Знамени', 'Тыл Совежеких Вооруженных',
                'Совета Народных Комиссаров', 'Трудового Краеного Знамени']

pattern_matches = [name.replace("\n", "") for name in pattern_matches]
pattern_matches = [name.replace("      ", "") for name in pattern_matches]
pattern_matches = [name for name in pattern_matches if name not in list_to_drop]
last_index = pattern_matches.index('Яшнов Борис Дмитриевич')
pattern_matches = pattern_matches[:last_index+1]

lurie_text = lurie_text.replace('\n', ' ')
lurie_text = lurie_text.replace(' \n', ' ')
lurie_text = lurie_text.replace('      ', ' ')

# для каждого имени достаем описание
for name in pattern_matches:
    print("______________")
    print(name)
    name_upper = name.replace('Арсеньев Николай Ваеильевич', 'Арсеньев Николай Ваcильевич')
    name_upper = name_upper.replace('Конставтин Михайлович Быков', 'Константин Михайлович Быков')
    name_upper = name_upper.replace('Васильев Виктор Консетантинович', 'Васильев Виктор Константинович')
    name_upper = name_upper.replace('Глинков Евгений Георгие', 'Глинков Евгений Георгиевич')
    name_upper = name_upper.replace('Марков Филипи Савельевич', 'Марков Филипп Савельевич')
    name_upper = name_upper.replace('Несвицкий Николай Никопаевич', 'Несвицкий Николай Николаевич')
    name_upper = name_upper.replace('Попов Борис Дмитриевич', 'Попов Борие Дмитриевич')
    name_upper = name_upper.replace('Почиковский Борисе Антонович', 'Почиковский Борис Антонович')
    name_upper = name_upper.replace('Нетров Борис Лаврентьевич', 'Петров Борис Лаврентьевич')
    name_upper = name_upper.replace('Хорошхин Борисе Владимирович', 'Хорошхин Борис Владимирович')
    name_upper = name_upper.replace('Шчейнберг Генрих Владимирович', 'Штейнберг Генрих Владимирович')
    name_upper = name_upper.replace('Рябов Иетр Ефимович', 'Рябов Петр Ефимович')
    name_upper = name_upper.split(" ")[0].upper() + " " + name_upper.split(" ")[1] + " " + name_upper.split(" ")[-1]
    # Поиск описания для каждого имени
    if (name_upper in list_names_4041) and (name != "Рябов Иетр Ефимович"):
        name_index = pattern_matches.index(name)
        next_index = name_index + 1
        name1 = pattern_matches[name_index]
        name2 = pattern_matches[next_index]
        print(name1)
        print(name2)
        name_description = lurie_text.split(name1)[1].split(name2)[0]
        data[name] = name_description
        
lurie_df = pd.DataFrame(list(data.items()), columns=['Name', 'Biography'])
lurie_df

In [ ]:
lurie_df.to_csv("lurie_df.csv", encoding="'utf-8-sig'")

# Командармы

In [ ]:
komandarmy_list = ""
for page in komandarmy.pages:
    komandarmy_list = komandarmy_list + page.extract_text()
komandarmy_list = komandarmy_list.replace("\t", " ")
komandarmy_list = komandarmy_list.replace("\n", " ")
komandarmy_list = komandarmy_list.replace("- ", "")
komandarmy_list = komandarmy_list.split("Основные сокращения")[0]
komandarmy_list = komandarmy_list.replace("настоящая фамилия, имя и отчество ХАНФЕРЯНЦ Арменак Артемович (7.01.1902 г., с. Б. Таглар ныне Гадрутского района в Нагорном Карабахе, Азербайджанская Республика — 18.4.1950 г.)",
                                         "(7.01.1902 г., с. Б. Таглар ныне Гадрутского района в Нагорном Карабахе, Азербайджанская Республика — 18.4.1950 г.), настоящая фамилия, имя и отчество Ханферянц Арменак Артемович")
komandarmy_list

In [ ]:
# Составление словаря, где ключи - имена, а значения - соответствующие описания
data = {}
pattern = re.compile(r'\b[А-ЯЁ]+\s+[А-ЯЁ][а-яё]+\s+[А-ЯЁ][а-яё]+')
pattern_matches = pattern.findall(komandarmy_list)

list_to_drop = ['В Красной Армии', 'В Великую Отечественную', 'В Великой Отечественной', 'ФЕДЕРАЦИИ Великая Отечественная',
               'ВДВ Красной Армии', 'ВВС Красной Армии', 'В Красной Лрмии', 'ПВО Красной Армии', 'ВВС Красной Ар',
               'В Красной Ар', 'ЦК Польской Объединенной', 'ВВС Войска Польского', 'ВОСО Красной Армии', 'ПВО Войск Дальнего',
               'Ь Петр Максимо', 'ГАУ Красной Армии', 'В Красной Армин', 'Ь Петр Максимоу']

pattern_matches = [name for name in pattern_matches if name not in list_to_drop]

# для каждого имени достаем описание
for name in pattern_matches:
    print("______________")
    print(name)
    # Поиск описания для каждого имени
    if name in list_names_4041 and name != "ХУДЯКОВ Сергей Александрович":
        name_index = pattern_matches.index(name)
        next_index = name_index + 1
        name1 = pattern_matches[name_index]
        name2 = pattern_matches[next_index]
        print(name1)
        print(name2)
        name_description = komandarmy_list.split(name1)[1].split(name2)[0]
        data[name] = name_description
    if name == "ХУДЯКОВ Сергей Александрович":
        name_description = komandarmy_list.split(name)[1]
        data[name] = name_description
        
comandarmy_df = pd.DataFrame(list(data.items()), columns=['Name', 'Biography'])
comandarmy_df

In [24]:
# Функция для извлечения даты рождения
def extract_birth_date1(content):
    match = re.search(r'\d{4}', content[:15])
    if match != None:
        return match.group()
    else:
        return None

# Функция для извлечения национальности
def extract_nationality(content):
    # Поиск национальности перед точкой с запятой
    match = re.search(r'\b[А-ЯЁ][а-яё]+\.', content)
    if match:
        return match.group()
    else:
        return None

# Пример использования этих функций
def process_dataframe(df):
    df['Birth Year'] = df['Biography'].apply(extract_birth_date1)
    df['Nationality'] = df['Biography'].apply(extract_nationality)
    return df

# Обработка DataFrame
processed_df = process_dataframe(comandarmy_df)
processed_df = processed_df.replace("Артемович.", "Армянин.")
processed_df = processed_df.reset_index()
processed_df = processed_df.drop("index", axis=1)
processed_df["Source"] = "Командармы"
processed_df

,Name,Biography,Birth Year,Nationality,Source
0,АКИМОВ Степан Дмитриевич,"(11.01. 1896 г., дер. Ханцевка ныне Псковской...",1896,Русский.,Командармы
1,АНТОНЮК Максим Антонович,"(19.10. 1895 г., дер. Мацы ныне Пружанского р...",1895,Белорус.,Командармы
2,БАГРАМЯН Иван Христофорович,"(2.12.1897 г., г. Елизаветполь — 21.9.1982 г....",1897,Армянин.,Командармы
3,БАТОВ Павел Иванович,"(1.6.1897 г., дер. Филисово ныне Рыбинского р...",1897,Русский.,Командармы
4,БАТЮНЯ Александр Григорьевич,"(2.6.1898 г., дер. Курганы ныне Кировского ра...",1898,Белорус.,Командармы
...,...,...,...,...,...
187,РЫБАЛЬЧЕНКО Степан Дмитриевич,"(11.7.1903 г., с. Новоегорлыкское ныне Сальск...",1903,Русский.,Командармы
188,СЕНАТОРОВ Александр Сергеевич,"(8.10.1912 г., с. Вышелсй ныне Городищенского...",1912,Русский.,Командармы
189,СОКОЛОВ Иван Михайлович,"(18.6. 1900 г., дер. Петраково ныне Старицког...",1900,Русский.,Командармы
190,ХРЮКИН Тимофей Тимофеевич,"(21.6. 1910 г., г. Ейск ныне Краснодарского к...",1910,Русский.,Командармы


In [25]:
# merge dataframes -> resulting dataframe "generals_intermediate_df.csv"
processed_df.to_csv("comandarmy_df.csv", encoding='utf-16')
data = pd.read_csv("comandarmy_df.csv", encoding='utf-16')
data1 = pd.read_csv("updated_lurie_df.csv", encoding='utf-8-sig')
data1["Source"] = "Лурье"
concat_df = pd.concat([data, data1], ignore_index = True)
concat_df.to_csv("generals_intermediate_df.csv", encoding='utf-16')
concat_df

,Unnamed: 0,Name,Biography,Birth Year,Nationality,Source
0,0.0,АКИМОВ Степан Дмитриевич,"(11.01. 1896 г., дер. Ханцевка ныне Псковской...",1896.0,Русский.,Командармы
1,1.0,АНТОНЮК Максим Антонович,"(19.10. 1895 г., дер. Мацы ныне Пружанского р...",1895.0,Белорус.,Командармы
2,2.0,БАГРАМЯН Иван Христофорович,"(2.12.1897 г., г. Елизаветполь — 21.9.1982 г....",1897.0,Армянин.,Командармы
3,3.0,БАТОВ Павел Иванович,"(1.6.1897 г., дер. Филисово ныне Рыбинского р...",1897.0,Русский.,Командармы
4,4.0,БАТЮНЯ Александр Григорьевич,"(2.6.1898 г., дер. Курганы ныне Кировского ра...",1898.0,Белорус.,Командармы
...,...,...,...,...,...,...
306,NaN,Хорошхин Борисе Владимирович,"[30.6(12.7).1892, С.-Петербург — 1.8.1942], с...",1892.0,Русский,Лурье
307,NaN,Шершов Александр Павлович,"[26.7(7.8).1874, С.-Петербург — 7.5.1958, Лен...",1874.0,Русский,Лурье
308,NaN,Шталь Александр Викторович,"[17(29).4.1865, С.-Петербург — 13.6.1950, Лен...",1865.0,Русский,Лурье
309,NaN,Юмашев Иван Степанович,"[27.9(9.10).1895, г. Тифлис, ныне г. Тбилиси,...",1895.0,Русский,Лурье


In [ ]:
pattern = re.compile(r'\b[А-ЯЁ]+\b\s+[А-ЯЁ][а-яё]+\s+[А-ЯЁ][а-яё]+')

komandarmy_list = ""
for page in komandarmy.pages:
    komandarmy_list = komandarmy_list + page.extract_text()

# Find all matches in the text
komandarmy_matches = pattern.findall(komandarmy_list)
komandarmy_matches = [match.replace("\t", " ").replace("\n", " ") for match in komandarmy_matches]
komandarmy_matches = [value for value in komandarmy_matches if value != 'В Красной Армии']
komandarmy_matches = [value for value in komandarmy_matches if value != 'В Великую Отечественную']
komandarmy_matches = [value for value in komandarmy_matches if value != 'В Великой Отечественной']
komandarmy_matches = [value for value in komandarmy_matches if value != 'ФЕДЕРАЦИИ Великая Отечественная']
komandarmy_matches = [value for value in komandarmy_matches if value != 'ВДВ Красной Армии']
komandarmy_matches = [value for value in komandarmy_matches if value != 'ВВС Красной Армии']
komandarmy_matches = [value for value in komandarmy_matches if value != 'В Красной Лрмии']
komandarmy_matches = [value for value in komandarmy_matches if value != 'ПВО Красной Армии']
komandarmy_matches = [value for value in komandarmy_matches if value != 'ВВС Красной Ар']
komandarmy_matches = [value for value in komandarmy_matches if value != 'В Красной Ар']
komandarmy_matches = [value for value in komandarmy_matches if value != 'ЦК Польской Объединенной']
komandarmy_matches = [value for value in komandarmy_matches if value != 'ВВС Войска Польского']
komandarmy_matches = [value for value in komandarmy_matches if value != 'ВОСО Красной Армии']
komandarmy_matches = [value for value in komandarmy_matches if value != 'ПВО Войск Дальнего']
komandarmy_matches = [value for value in komandarmy_matches if value != 'Ь Петр Максимо']
komandarmy_matches = [value for value in komandarmy_matches if value != 'ГАУ Красной Армии']


counter = 0
komandarmy_matches_list = []
for name in komandarmy_matches:
    if name in list_names_4041:
        #print(name)
        counter +=1
        komandarmy_matches_list.append(name)

print(f"Matches in Komandarmy: {counter}")

# Комкоры

In [26]:
import re

pattern = re.compile(r'\b[А-ЯЁ]+\b [А-ЯЁ][а-яё]+ [А-ЯЁ][а-яё]+')

# Find all matches in the text
matches = pattern.findall(komcory)
matches = [value for value in matches if value != 'В Великую Отечественную']
matches = [value for value in matches if value != 'В Великую Отечествен']
matches = [value for value in matches if value != 'В Великую Отече']
matches = [value for value in matches if value != 'В Красной Армии']
matches = [value for value in matches if value != 'ВВС Красной Армии']
matches = [value for value in matches if value != 'В Красной Гвардии']
matches = [value for value in matches if value != 'ЦИК Крымской Республики']
matches = [value for value in matches if value != 'СССР Маршала Советского']
matches = [value for value in matches if value != 'КУНС Отдельной Кавказской']
matches = [value for value in matches if value != 'ВДВ Красной Армии']
matches = [value for value in matches if value != 'В Краснои Армии']
matches = [value for value in matches if value != 'ГУК Советской Армии']
matches = [value for value in matches if value != 'ВДВ Советской Армии']
matches = [value for value in matches if value != 'РККА Кавказской Краснознаменной']
matches = [value for value in matches if value != 'В Великой Отечественной']
matches = [value for value in matches if value != 'ПВО Красной Армии']
matches = [value for value in matches if value != 'АБТВ Красной Армии']
matches = [value for value in matches if value != 'ВВС Советской Армии']
matches = [value for value in matches if value != 'ГУК Вооруженных Сил']
matches = [value for value in matches if value != 'ГУК Красной Армии']
komcory_matches1 = matches


matches = pattern.findall(komcory2)
#matches.remove('В Великую Отечественную')
matches = [value for value in matches if value != 'В Великую Отечественную']
matches = [value for value in matches if value != 'В Великую Отечествен']
matches = [value for value in matches if value != 'В Великую Отече']
matches = [value for value in matches if value != 'В Красной Армии']
matches = [value for value in matches if value != 'ВВС Красной Армии']
matches = [value for value in matches if value != 'В Красной Гвардии']
matches = [value for value in matches if value != 'ЦИК Крымской Республики']
matches = [value for value in matches if value != 'СССР Маршала Советского']
matches = [value for value in matches if value != 'КУНС Отдельной Кавказской']
matches = [value for value in matches if value != 'ВДВ Красной Армии']
matches = [value for value in matches if value != 'В Краснои Армии']
matches = [value for value in matches if value != 'ГУК Советской Армии']
matches = [value for value in matches if value != 'ВДВ Советской Армии']
matches = [value for value in matches if value != 'РККА Кавказской Краснознаменной']
matches = [value for value in matches if value != 'В Великой Отечественной']
matches = [value for value in matches if value != 'ПВО Красной Армии']
matches = [value for value in matches if value != 'АБТВ Красной Армии']
matches = [value for value in matches if value != 'ВВС Советской Армии']
matches = [value for value in matches if value != 'ГУК Вооруженных Сил']
matches = [value for value in matches if value != 'ГУК Красной Армии']
matches = [value for value in matches if value != 'АБТУ Красной Армии']
matches = [value for value in matches if value != 'МБ Советской Армии']
matches = [value for value in matches if value != 'АБТВ Кавказской Красной']
matches = [value for value in matches if value != 'БТВ Красной Армии']
matches = [value for value in matches if value != 'ГАБТУ Красной Армии']
matches = [value for value in matches if value != 'ЧОН Советской Социалистической']
matches = [value for value in matches if value != 'КУКС Красной Армии']
matches = [value for value in matches if value != 'ПВО Отдельной Кавказской']
matches = [value for value in matches if value != 'ВВС Дальнего Востока']
matches = [value for value in matches if value != 'ПВО Китайской Народной']
matches = [value for value in matches if value != 'ВВС Венгерской Народной']
matches = [value for value in matches if value != 'ВВС Отдельной Кавказской']
matches = [value for value in matches if value != 'АДД Красной Армии']
matches = [value for value in matches if value != 'ВВС Войска Польского']
komcory_matches2 = matches


print(f"Names in Komcory p.1: {len(komcory_matches1)}")
print(f"Names in Komcory p.2: {len(komcory_matches2)}")
print(f"Total number of names in komcory (Soloviev): {len(komcory_matches1) + len(komcory_matches2)}")

komcory_matches_all = komcory_matches1 + komcory_matches2

komcory_matches = []
counter = 0
for name in komcory_matches_all:
    if name in list_names_4041:
        #print(name)
        counter +=1
        komcory_matches.append(name)

print(f"Matches in Komcory p1: {counter}")

Names in Komcory p.1: 892
Names in Komcory p.2: 634
Total number of names in komcory (Soloviev): 1526
Matches in Komcory p1: 349


In [ ]:
komcory = komcory.split("\n\nСодержание")[0]
komcory = komcory.replace("\t", " ")
komcory = komcory.replace("\n", " ")
komcory = komcory.replace("- ", "")
komcory

In [ ]:
# Составление словаря, где ключи - имена, а значения - соответствующие описания
data = {}
pattern = re.compile(r'\b[А-ЯЁ]+\s+[А-ЯЁ][а-яё]+\s+[А-ЯЁ][а-яё]+')
pattern_matches = pattern.findall(komcory)
list_to_drop = ['В Красной Армии', 'В Великую Отечественную', 'В Великой Отечественной', 'ФЕДЕРАЦИИ Великая Отечественная',
               'ВДВ Красной Армии', 'ВВС Красной Армии', 'В Красной Лрмии', 'ПВО Красной Армии', 'ВВС Красной Ар',
               'В Красной Ар', 'ЦК Польской Объединенной', 'ВВС Войска Польского', 'ВОСО Красной Армии', 'ПВО Войск Дальнего',
               'Ь Петр Максимо', 'ГАУ Красной Армии', 'В Красной Армин', 'Ь Петр Максимоу', 'КУНС Отдельной Кавказской',
               'В Краснои Армии', 'ГУК Советской Армии', 'ВДВ Советской Армии', 'РККА Кавказской Краснознаменной', 
               'СССР Маршала Советского', 'ГУК Советской Армии', 'ГУК Советской Армии', 'ВВС Советской Армии', 'АБТВ Красной Армии',
               'ГУК Советской Армии', 'ГУК Вооруженных Сил', 'СССР Маршала Советского', ]

pattern_matches = [name for name in pattern_matches if name not in list_to_drop]

# для каждого имени достаем описание
for name in pattern_matches:
    print("______________")
    print(name)
    # Поиск описания для каждого имени
    if name in list_names_4041 and name != "ЯНОВСКИЙ Александр Яковлевич":
        name_index = pattern_matches.index(name)
        next_index = name_index + 1
        name1 = pattern_matches[name_index]
        name2 = pattern_matches[next_index]
        print(name1)
        print(name2)
        name_description = komcory.split(name1)[1].split(name2)[0]
        data[name] = name_description
    if name == "ЯНОВСКИЙ Александр Яковлевич":
        name_description = komcory.split(name)[1]
        data[name] = name_description
        
komcory_df = pd.DataFrame(list(data.items()), columns=['Name', 'Biography'])
komcory_df

In [ ]:
komcory2 = komcory2.split("\n\nОсновные сокращения, принятые в словаре\n\n")[0]
komcory2 = komcory2.replace("\t", " ")
komcory2 = komcory2.replace("\n", " ")
komcory2 = komcory2.replace("- ", "")
komcory2

In [ ]:
# Составление словаря, где ключи - имена, а значения - соответствующие описания
data = {}
pattern = re.compile(r'\b[А-ЯЁ]+\s+[А-ЯЁ][а-яё]+\s+[А-ЯЁ][а-яё]+')
pattern_matches = pattern.findall(komcory2)
list_to_drop = ['В Красной Армии', 'В Великую Отечественную', 'В Великой Отечественной', 'ФЕДЕРАЦИИ Великая Отечественная',
               'ВДВ Красной Армии', 'ВВС Красной Армии', 'В Красной Лрмии', 'ПВО Красной Армии', 'ВВС Красной Ар',
               'В Красной Ар', 'ЦК Польской Объединенной', 'ВВС Войска Польского', 'ВОСО Красной Армии', 'ПВО Войск Дальнего',
               'Ь Петр Максимо', 'ГАУ Красной Армии', 'В Красной Армин', 'Ь Петр Максимоу', 'КУНС Отдельной Кавказской',
               'В Краснои Армии', 'ГУК Советской Армии', 'ВДВ Советской Армии', 'РККА Кавказской Краснознаменной', 
               'СССР Маршала Советского', 'ГУК Советской Армии', 'ГУК Советской Армии', 'ВВС Советской Армии', 'АБТВ Красной Армии',
               'ГУК Советской Армии', 'ГУК Вооруженных Сил', 'СССР Маршала Советского', ]

pattern_matches = [name for name in pattern_matches if name not in list_to_drop]

# для каждого имени достаем описание
for name in pattern_matches:
    print("______________")
    print(name)
    # Поиск описания для каждого имени
    if name in list_names_4041 and name != "ПРОСВИРОВ Никон Андреевич":
        name_index = pattern_matches.index(name)
        next_index = name_index + 1
        name1 = pattern_matches[name_index]
        name2 = pattern_matches[next_index]
        print(name1)
        print(name2)
        name_description = komcory2.split(name1)[1].split(name2)[0]
        data[name] = name_description
    if name == "ПРОСВИРОВ Никон Андреевич":
        name_description = komcory2.split(name)[1]
        data[name] = name_description
        
komcory_df2 = pd.DataFrame(list(data.items()), columns=['Name', 'Biography'])
komcory_df2

In [85]:
# Функция для извлечения даты рождения
def extract_birth_date1(content):
    match = re.search(r'\d{4}', content[:15])
    if match != None:
        return match.group()
    else:
        return None

# Функция для извлечения национальности
def extract_nationality(content):
    # Поиск национальности перед точкой с запятой
    match = re.search(r'\b[А-ЯЁ][а-яё]+\.', content)
    if match:
        return match.group()
    else:
        return None

# Пример использования этих функций
def process_dataframe(df):
    df['Birth Year'] = df['Biography'].apply(extract_birth_date1)
    df['Nationality'] = df['Biography'].apply(extract_nationality)
    return df

# Обработка DataFrame
processed_df2 = process_dataframe(komcory_df2)
processed_df2 = processed_df2.reset_index()
processed_df2 = processed_df2.drop("index", axis=1)
processed_df2["Source"] = "Комкоры"
processed_df2

,Name,Biography,Birth Year,Nationality,Source
0,БЕЗУГЛЫЙ Иван Семенович,"(23.10.1897 г., с. Заерок ныне Старобельского...",1897,Украинец.,Комкоры
1,ГЛАЗУНОВ Василий Афанасьевич,"(1.1. 1896 г., дер. Варваровка нынеСердобског...",1896,Русский.,Комкоры
2,ИВАНОВ Николай Петрович,"(6.12. 1897 г., Москва — 25.2.1943 г., погиб ...",1897,Русский.,Комкоры
3,КИРЗИМОВ Александр Ильич,"(12.12. 1897 г., г. Верный, ныне г. Алма-Ата,...",1897,Татарин.,Комкоры
4,ЛЯПИН Петр Иванович,"(25.6.1894 г., с. Ромоданово, ныне пгт Ромод...",1894,Русский.,Комкоры
...,...,...,...,...,...
72,СПИРИН Иван Тимофеевич,"(9.8.1898 г., г. Коломна Московской губ. — 4....",1898,Русский.,Комкоры
73,ТУРКЕЛЬ Иван Лукич,"(9.10.1903 г., деревня Федоровка Александровс...",1903,Украинец.,Комкоры
74,ШЕВЧЕНКО Владимир Илларионович,"(25.7.1908 г., пос. Панфилово Новоаннинского...",1908,Украинец.,Комкоры
75,ГОЛОВКО Владимир Александрович,"(25.02.1895 г., г. Томск — 30.5.1956 г., Моск...",1895,Русский.,Комкоры


In [86]:
interm_df = pd.read_csv("generals_intermediate_df.csv", encoding='utf-16')
interm_df

,Unnamed: 0.1,Unnamed: 0,Name,Biography,Birth Year,Nationality,Source
0,0,0.0,АКИМОВ Степан Дмитриевич,"(11.01. 1896 г., дер. Ханцевка ныне Псковской...",1896.0,Русский.,Командармы
1,1,1.0,АНТОНЮК Максим Антонович,"(19.10. 1895 г., дер. Мацы ныне Пружанского р...",1895.0,Белорус.,Командармы
2,2,2.0,БАГРАМЯН Иван Христофорович,"(2.12.1897 г., г. Елизаветполь — 21.9.1982 г....",1897.0,Армянин.,Командармы
3,3,3.0,БАТОВ Павел Иванович,"(1.6.1897 г., дер. Филисово ныне Рыбинского р...",1897.0,Русский.,Командармы
4,4,4.0,БАТЮНЯ Александр Григорьевич,"(2.6.1898 г., дер. Курганы ныне Кировского ра...",1898.0,Белорус.,Командармы
...,...,...,...,...,...,...,...
306,306,NaN,Хорошхин Борисе Владимирович,"[30.6(12.7).1892, С.-Петербург — 1.8.1942], с...",1892.0,Русский,Лурье
307,307,NaN,Шершов Александр Павлович,"[26.7(7.8).1874, С.-Петербург — 7.5.1958, Лен...",1874.0,Русский,Лурье
308,308,NaN,Шталь Александр Викторович,"[17(29).4.1865, С.-Петербург — 13.6.1950, Лен...",1865.0,Русский,Лурье
309,309,NaN,Юмашев Иван Степанович,"[27.9(9.10).1895, г. Тифлис, ныне г. Тбилиси,...",1895.0,Русский,Лурье


In [87]:
concat_df = pd.concat([interm_df, processed_df1, processed_df2], ignore_index = True)
concat_df = concat_df.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1)
concat_df

,Name,Biography,Birth Year,Nationality,Source
0,АКИМОВ Степан Дмитриевич,"(11.01. 1896 г., дер. Ханцевка ныне Псковской...",1896.0,Русский.,Командармы
1,АНТОНЮК Максим Антонович,"(19.10. 1895 г., дер. Мацы ныне Пружанского р...",1895.0,Белорус.,Командармы
2,БАГРАМЯН Иван Христофорович,"(2.12.1897 г., г. Елизаветполь — 21.9.1982 г....",1897.0,Армянин.,Командармы
3,БАТОВ Павел Иванович,"(1.6.1897 г., дер. Филисово ныне Рыбинского р...",1897.0,Русский.,Командармы
4,БАТЮНЯ Александр Григорьевич,"(2.6.1898 г., дер. Курганы ныне Кировского ра...",1898.0,Белорус.,Командармы
...,...,...,...,...,...
483,СПИРИН Иван Тимофеевич,"(9.8.1898 г., г. Коломна Московской губ. — 4....",1898,Русский.,Комкоры
484,ТУРКЕЛЬ Иван Лукич,"(9.10.1903 г., деревня Федоровка Александровс...",1903,Украинец.,Комкоры
485,ШЕВЧЕНКО Владимир Илларионович,"(25.7.1908 г., пос. Панфилово Новоаннинского...",1908,Украинец.,Комкоры
486,ГОЛОВКО Владимир Александрович,"(25.02.1895 г., г. Томск — 30.5.1956 г., Моск...",1895,Русский.,Комкоры


In [88]:
concat_df.to_csv("generals_intermediate_df2.csv", encoding='utf-16')

### Комдивы

In [34]:
komdivy_names = pd.read_csv("comdivy_spisok_imen.csv")
komdivy_names = komdivy_names.rename({'КОМАНДИРЫ МОТОСТРЕЛКОВЫХ И МОТОРИЗОВАННЫХ ДИВИЗИЙ': 'name'}, axis=1)
print(f"Total numder of Komdivy names (website) : {len(komdivy_names)}")

counter = 0
komdivy_matches_website = []
for name in list(komdivy_names['name']):
    if name in list_names_4041:
        counter +=1
        komdivy_matches_website.append(name)

print(f"Matches in Komdivy (website): {counter}")

Total numder of Komdivy names (website) : 3172
Matches in Komdivy (website): 334


БИРИЧЕВ Иван Иванович - Командиры стрелковых дивизий
КАЛИНИН Николай Васильевич - Командиры стрелковых дивизий

In [90]:
komdivy_text1 = ""
for page in komdivy_p1.pages:
    komdivy_text1 = komdivy_text1 + page.extract_text()

komdivy_text2 = ""
for page in komdivy_p2.pages:
    komdivy_text2 = komdivy_text2 + page.extract_text()

komdivy_text3 = ""
for page in komdivy_p3.pages:
    komdivy_text3 = komdivy_text3 + page.extract_text()

# fing pattern through regex
pattern = re.compile(r'\b[А-ЯЁ]+\b [А-ЯЁ][а-яё]+ [А-ЯЁ][а-яё]+')

# Find all matches in the text
matches = pattern.findall(komdivy_text1)
matches = [value for value in matches if value != 'В Великую Отечественную']
matches = [value for value in matches if value != 'В Великую Отечествен']
matches = [value for value in matches if value != 'В Великую Отече']
matches = [value for value in matches if value != 'ВВС Красной Армии']
matches = [value for value in matches if value != 'В Красной Гвардии']
matches = [value for value in matches if value != 'ЦИК Крымской Республики']
matches = [value for value in matches if value != 'СССР Маршала Советского']
matches.remove('КУКС Красной Армии')
matches.remove('В Красной Армии')
matches.remove('В Украинской Галицийской')
matches.remove('ГОУ Генштаба Советской')
matches.remove('ВО Войска Польского')
matches.remove('ГРУ Генштаба Красной')
matches.remove('ПВО Войск Дальнего')
matches.remove('В Армянской Красной')
matches.remove('ВЧК Средней Азии')
matches.remove('ПВО Красной Армии')
komdivy_matches1 = matches

matches = pattern.findall(komdivy_text2)
matches = [value for value in matches if value != 'В Великую Отечественную']
matches = [value for value in matches if value != 'В Великую Отечествен']
matches = [value for value in matches if value != 'В Великую Оте']
matches = [value for value in matches if value != 'В Красной Армии']
matches = [value for value in matches if value != 'В Красной Гвардии']
matches = [value for value in matches if value != 'СССР Маршала Советского']
matches = [value for value in matches if value != 'РУ Генштаба Красной']
matches = [value for value in matches if value != 'ВО Войска Польского']
matches = [value for value in matches if value != 'ВВС Краснознаменного Балтийского']
matches = [value for value in matches if value != 'ГУ Красной Армии']
matches = [value for value in matches if value != 'У Ра Юго']
matches.remove('РФ Герой Советского')
matches.remove('ОГПУ Северного Кавказа')
komdivy_matches2 = matches

matches = pattern.findall(komdivy_text3)
matches = [value for value in matches if value != 'В Великую Отечественную']
matches = [value for value in matches if value != 'В Великую Отечествен']
matches = [value for value in matches if value != 'В Великую Оте']
matches = [value for value in matches if value != 'В Красной Армии']
matches = [value for value in matches if value != 'В Красной Гвардии']
matches = [value for value in matches if value != 'СССР Маршала Советского']
matches = [value for value in matches if value != 'РУ Генштаба Красной']
matches = [value for value in matches if value != 'ВО Войска Польского']
matches = [value for value in matches if value != 'ВВС Краснознаменного Балтийского']
matches = [value for value in matches if value != 'ГУ Красной Армии']
matches = [value for value in matches if value != 'У Ра Юго']
matches = [value for value in matches if value != 'ГОУ Генштаба Советской']
matches = [value for value in matches if value != 'ОГПУ Средней Азии']
komdivy_matches3 = matches

print(f"Names in Komdivy p.1: {len(komdivy_matches1)}")
print(f"Names in Komdivy p.2: {len(komdivy_matches2)}")
print(f"Names in Komdivy p.3: {len(komdivy_matches3)}")
print(f"Total number of names in komdivy (Soloviev): {len(komdivy_matches1) + len(komdivy_matches2) + len(komdivy_matches3)}")

komdivy_matches_all = komdivy_matches1 + komdivy_matches2 + komdivy_matches3

komdivy_matches = []
counter = 0
for name in komdivy_matches_all:
    if name in list_names_4041:
        #print(name)
        counter +=1
        komdivy_matches.append(name)

print(f"Matches in Komdivy p1: {counter}")

Names in Komdivy p.1: 771
Names in Komdivy p.2: 558
Names in Komdivy p.3: 480
Total number of names in komdivy (Soloviev): 1809
Matches in Komdivy p1: 205


In [ ]:
komdivy_text1 = komdivy_text1.split('Денис \xa0Соловьев\nКомандиры дивизий Красной\nАрмии 1941-1945 гг. \
Том 1\n«ЛитРес: Самиздат»\n2020Соловьев\xa0Д.\xa0Ю.\nКомандиры дивизий Красной Армии 1941-1945 гг. \
Том 1 \xa0/\xa0\nД.\xa0Ю.\xa0Соловьев\xa0—\xa0 «ЛитРес: Самиздат»,\xa0 2020\nISBN 978-5-532-06250-4\nВ многотомнике \
"Командиры дивизий Красной Армии 1941-1945 гг." собраны\nбиографии всех командиров и временно исполняющих должности \
командиров\nдивизий Красной Армии с 1941 года по 1945 год. Биографии составлены по\nданным УПК и личных дел генералов и \
офицеров Красной Армии. В 1-й том\nвошли биографии командиров горнострелковых, мотострелковых, стрелковых\nи дивизий \
народного ополчения с фамилиями на букву А (часть 1).\nISBN 978-5-532-06250-4 ©\xa0Соловьев\xa0Д.\xa0Ю.,\xa02020\n©\xa0ЛитРес: \
Самиздат,\xa02020Д.\xa0 Ю.\xa0 Соловьев.\xa0 «Командиры дивизий Красной Армии 1941-1945 гг. Том 1»\n4')[1]

komdivy_text2 = komdivy_text2.split('Денис \xa0Соловьев\nКомандиры дивизий Красной\nАрмии 1941-1945 гг. Том 31\n«ЛитРес: \
Самиздат»\n2020Соловьев\xa0Д.\xa0Ю.\nКомандиры дивизий Красной Армии 1941-1945 гг. \
Том 31 \xa0/\xa0\nД.\xa0Ю.\xa0Соловьев\xa0—\xa0 «ЛитРес: Самиздат»,\xa0 2020\nISBN 978-5-532-05857-6\nВ многотомнике \
"Командиры дивизий Красной Армии 1941-1945 гг." собраны\nбиографии всех командиров и временно исполняющих должности командиров\nдивизий Красной Армии \
с 1941 года по 1945 год. Биографии составлены по\nданным УПК и личных дел генералов и офицеров Красной Армии. \
В 31-й том\nвошли биографии командиров горнострелковых, мотострелковых, стрелковых\nи дивизий народного ополчения \
с фамилиями на букву К (часть 6).\nISBN 978-5-532-05857-6 ©\xa0Соловьев\xa0Д.\xa0Ю.,\xa02020\n©\xa0ЛитРес: \
Самиздат,\xa02020Д.\xa0 Ю.\xa0 Соловьев.\xa0 «Командиры дивизий Красной Армии 1941-1945 гг. Том 31»\n4')[1]


komdivy_text3 = komdivy_text3.split('Денис \xa0Соловьев\nКомандиры дивизий Красной\nАрмии 1941-1945 гг. Том 56\n«ЛитРес: \
Самиздат»\n2020Соловьев\xa0Д.\xa0Ю.\nКомандиры дивизий Красной Армии 1941-1945 гг. \
Том 56 \xa0/\xa0\nД.\xa0Ю.\xa0Соловьев\xa0—\xa0 «ЛитРес: Самиздат»,\xa0 2020\nВ многотомнике \
"Командиры дивизий Красной Армии 1941-1945 гг." собраны\nбиографии всех командиров и временно исполняющих должности \
командиров\nдивизий Красной Армии с 1941 года по 1945 год. Биографии составлены по\nданным УПК и личных дел генералов \
и офицеров Красной Армии. В 56-й том\nвошли биографии командиров горнострелковых, мотострелковых, стрелковых\nи дивизий \
народного ополчения с фамилиями на букву Р.\n©\xa0Соловьев\xa0Д.\xa0Ю.,\xa02020\n©\xa0ЛитРес: \
Самиздат,\xa02020Д.\xa0 Ю.\xa0 Соловьев.\xa0 «Командиры дивизий Красной Армии 1941-1945 гг. Том 56»\n')[1]

komdivy = komdivy_text1 + komdivy_text2 + komdivy_text3
komdivy = komdivy.replace("\t", " ")
komdivy = komdivy.replace("\n", " ")
komdivy = komdivy.replace("- ", "")
komdivy = komdivy.replace("\xa0", "")
komdivy

In [ ]:
# Составление словаря, где ключи - имена, а значения - соответствующие описания
data = {}
pattern = re.compile(r'\b[А-ЯЁ]+\s+[А-ЯЁ][а-яё]+\s+[А-ЯЁ][а-яё]+')
pattern_matches = pattern.findall(komdivy)
list_to_drop = ['В Красной Армии', 'В Великую Отечественную', 'В Великой Отечественной', 'ФЕДЕРАЦИИ Великая Отечественная',
               'ВДВ Красной Армии', 'ВВС Красной Армии', 'В Красной Лрмии', 'ПВО Красной Армии', 'ВВС Красной Ар',
               'В Красной Ар', 'ЦК Польской Объединенной', 'ВВС Войска Польского', 'ВОСО Красной Армии', 'ПВО Войск Дальнего',
               'Ь Петр Максимо', 'ГАУ Красной Армии', 'В Красной Армин', 'Ь Петр Максимоу', 'КУНС Отдельной Кавказской',
               'В Краснои Армии', 'ГУК Советской Армии', 'ВДВ Советской Армии', 'РККА Кавказской Краснознаменной', 
               'СССР Маршала Советского', 'ГУК Советской Армии', 'ГУК Советской Армии', 'ВВС Советской Армии', 'АБТВ Красной Армии',
               'ГУК Советской Армии', 'ГУК Вооруженных Сил', 'СССР Маршала Советского', 'КУКС Красной Армии', 'В Красной Гвардии', 
               'В Украинской Галицийской', 'ГОУ Генштаба Советской', 'В Красной Гвардии', 'ЧОН Восточной Сибири', 'СССР Маршале Советского',
               'ВО Войска Польского', 'ОГПУ Средней Азии', 'В Северного Кавказа', 'В Армянской Красной', 'ВЧК Средней Азии',
                'РФ Герой Советского', 'ОГПУ Северного Кавказа', 'РУ Генштаба Красной', 'ВВС Краснознаменного Балтийского',
               'ГУ Красной Армии', 'У Ра Юго', 'ГРУ Генштаба Красной']

pattern_matches = [name for name in pattern_matches if name not in list_to_drop]

# для каждого имени достаем описание
for name in pattern_matches:
    print("______________")
    print(name)
    # Поиск описания для каждого имени
    if name in list_names_4041:
        name_index = pattern_matches.index(name)
        next_index = name_index + 1
        name1 = pattern_matches[name_index]
        name2 = pattern_matches[next_index]
        print(name1)
        print(name2)
        name_description = komdivy.split(name1)[1].split(name2)[0]
        data[name] = name_description
    #if name == "ПРОСВИРОВ Никон Андреевич":
        #name_description = komcory2.split(name)[1]
        #data[name] = name_description
        
komdivy_df = pd.DataFrame(list(data.items()), columns=['Name', 'Biography'])
komdivy_df

In [102]:
# Функция для извлечения даты рождения
def extract_birth_date1(content):
    match = re.search(r'\d{4}', content)
    return match.group()

# Функция для извлечения национальности
def extract_nationality(content):
    # Поиск национальности перед точкой с запятой
    match = re.search(r'\b[А-ЯЁ][а-яё]+\.', content)
    if match:
        return match.group()
    else:
        return None

# Пример использования этих функций
def process_dataframe(df):
    df['Birth Year'] = df['Biography'].apply(extract_birth_date1)
    df['Nationality'] = df['Biography'].apply(extract_nationality)
    return df

# Обработка DataFrame
processed_df3 = process_dataframe(komdivy_df)
processed_df3 = processed_df3.reset_index()
processed_df3 = processed_df3.drop("index", axis=1)
processed_df3["Source"] = "Комдивы"
processed_df3

,Name,Biography,Birth Year,Nationality,Source
0,АБРАМИДЗЕ Павел Ивлианович,Советский военачальник. Родился 19 марта 1901...,1901,Грузия.,Комдивы
1,АВВАКУМОВ Яков Александрович,Советский военачальник. Родился 1 октября 189...,1897,Русский.,Комдивы
2,АЛАВЕРДОВ Христофор Николаевич,Советский военачальник. Родился 25 мая 1895 г...,1895,Армянин.,Комдивы
3,АЛЕКСАНДРОВ Петр Алексеевич,Советский военачальник. Родился 26 июня 1899 ...,1899,Петербурге.,Комдивы
4,АЛЯБУШЕВ Филипп Федорович,Советский военачальник. Родился 13.11.1893 г....,1893,Русский.,Комдивы
...,...,...,...,...,...
199,ШИЛОВ Федор Николаевич,"Советский военачальник. (19.09.1904, дер. Слу...",1904,Русский.,Комдивы
200,ЩЕКОТСКИЙ Федор Михайлович,"Советский военачальник. (11.11.1898, пос. Доб...",1898,Украинец.,Комдивы
201,ЩЕРБАКОВ Владимир Иванович,"Советский военачальник. (27.07.1901, дер. Уры...",1901,Русский.,Комдивы
202,ЩЕРБАКОВ Николай Тимофеевич,"Советский военачальник. (01.05.1897, г. Москв...",1897,Русский.,Комдивы


In [104]:
concat_df = pd.read_csv("generals_intermediate_df2.csv", encoding='utf-16')
full_df = pd.concat([concat_df, processed_df3], ignore_index = True)
full_df = full_df.drop("Unnamed: 0", axis=1)
full_df

,Name,Biography,Birth Year,Nationality,Source
0,АКИМОВ Степан Дмитриевич,"(11.01. 1896 г., дер. Ханцевка ныне Псковской...",1896.0,Русский.,Командармы
1,АНТОНЮК Максим Антонович,"(19.10. 1895 г., дер. Мацы ныне Пружанского р...",1895.0,Белорус.,Командармы
2,БАГРАМЯН Иван Христофорович,"(2.12.1897 г., г. Елизаветполь — 21.9.1982 г....",1897.0,Армянин.,Командармы
3,БАТОВ Павел Иванович,"(1.6.1897 г., дер. Филисово ныне Рыбинского р...",1897.0,Русский.,Командармы
4,БАТЮНЯ Александр Григорьевич,"(2.6.1898 г., дер. Курганы ныне Кировского ра...",1898.0,Белорус.,Командармы
...,...,...,...,...,...
687,ШИЛОВ Федор Николаевич,"Советский военачальник. (19.09.1904, дер. Слу...",1904,Русский.,Комдивы
688,ЩЕКОТСКИЙ Федор Михайлович,"Советский военачальник. (11.11.1898, пос. Доб...",1898,Украинец.,Комдивы
689,ЩЕРБАКОВ Владимир Иванович,"Советский военачальник. (27.07.1901, дер. Уры...",1901,Русский.,Комдивы
690,ЩЕРБАКОВ Николай Тимофеевич,"Советский военачальник. (01.05.1897, г. Москв...",1897,Русский.,Комдивы


In [105]:
full_df.to_csv("generals_intermediate_df3.csv", encoding='utf-16')

In [115]:
def get_name_upper(name):
    return name.split(" ")[0].upper() + " " + name.split(" ")[1] + " " + name.split(" ")[-1]
    
full_df = pd.read_excel("generals_intermediate_df3.xls")
full_df['Name'] = full_df['Name'].apply(get_name_upper)
full_df

,Unnamed: 0,Name,Biography,Birth Year,Nationality,Source
0,0,АКИМОВ Степан Дмитриевич,"(11.01. 1896 г., дер. Ханцевка ныне Псковской...",1896.0,Русский.,Командармы
1,1,АНТОНЮК Максим Антонович,"(19.10. 1895 г., дер. Мацы ныне Пружанского р...",1895.0,Белорус.,Командармы
2,2,БАГРАМЯН Иван Христофорович,"(2.12.1897 г., г. Елизаветполь — 21.9.1982 г....",1897.0,Армянин.,Командармы
3,3,БАТОВ Павел Иванович,"(1.6.1897 г., дер. Филисово ныне Рыбинского р...",1897.0,Русский.,Командармы
4,4,БАТЮНЯ Александр Григорьевич,"(2.6.1898 г., дер. Курганы ныне Кировского ра...",1898.0,Белорус.,Командармы
...,...,...,...,...,...,...
684,687,ШИЛОВ Федор Николаевич,"Советский военачальник. (19.09.1904, дер. Слу...",1904.0,Русский.,Комдивы
685,688,ЩЕКОТСКИЙ Федор Михайлович,"Советский военачальник. (11.11.1898, пос. Доб...",1898.0,Украинец.,Комдивы
686,689,ЩЕРБАКОВ Владимир Иванович,"Советский военачальник. (27.07.1901, дер. Уры...",1901.0,Русский.,Комдивы
687,690,ЩЕРБАКОВ Николай Тимофеевич,"Советский военачальник. (01.05.1897, г. Москв...",1897.0,Русский.,Комдивы


In [116]:
full_df.to_csv("generals_intermediate_df3.csv", encoding='utf-16')

In [ ]:
full_df = pd.read_excel("generals_intermediate_df3.xls")

# Репрессии - основная таблица

In [ ]:
repressions_main_table['name'] = repressions_main_table["lname"] + " " + repressions_main_table["fname"] + " " + repressions_main_table["mname"]

repressions_matches_all = repressions_main_table["name"]

repressions_matches = []
counter = 0
for name in repressions_matches_all:
    if name in list_names_4041:
        #print(name)
        counter +=1
        repressions_matches.append(name)

print(f"Matches in Repressions main table: {counter}")

# Тархова

In [ ]:
tarkhova = PyPDF2.PdfReader('tarkhova-1.pdf')

tarkhova_list = ""
for page in tarkhova.pages:
    text = page.extract_text()
    tarkhova_list = tarkhova_list + text

pattern_trakhova = re.compile(r'\b[А-ЯЁ][а-яё]+\b [А-ЯЁ][а-яё]+ [А-ЯЁ][а-яё]+')
# Find all matches in the text
matches = pattern_trakhova.findall(tarkhova_list)
matches = [value.split(" ")[0].upper() + " " + value.split(" ")[1]+ " " + value.split(" ")[-1] for value in matches]
tarkhova_matches = matches

tarkhova_matches_list = []
counter = 0
for name in tarkhova_matches:
    if name in list_names_4041:
        counter +=1
        tarkhova_matches_list.append(name)

print(f"Matches in Tarkhova: {counter}")

# Петров Скоркин

In [117]:
petrov_skorkin = PyPDF2.PdfReader('petrov_skorkin.pdf')


petrov_skorkin_list = ""
for page in petrov_skorkin.pages:
    text = page.extract_text()
    petrov_skorkin_list = petrov_skorkin_list + text


pattern_petrov_skorkin = re.compile(r'\b[А-ЯЁ]+\b [А-ЯЁ]+ [А-ЯЁ]+')
# Find all matches in the text
matches = pattern_petrov_skorkin.findall(petrov_skorkin_list)
matches = [value.split(" ")[0] + " " + value.split(" ")[1].lower().capitalize() + " " + value.split(" ")[-1].lower().capitalize() for value in matches]
petrov_skorkin_matches = matches

petrov_skorkin_matches_list = []
counter = 0
for name in petrov_skorkin_matches:
    if name in list_names_4041:
        counter +=1
        petrov_skorkin_matches_list.append(name)

print(f"Matches in Petrov Skorkin: {counter}")

Matches in Petrov Skorkin: 9


### Залесский

In [118]:
zalessky = PyPDF2.PdfReader('zalessky.pdf')

zalessky_list = ""
for page in zalessky.pages:
    text = page.extract_text()
    zalessky_list = zalessky_list + text

pattern_zalessky = re.compile(r'\b[А-ЯЁ]+\b [А-ЯЁ][а-яё]+ [А-ЯЁ][а-яё]+')
# Find all matches in the text
matches = pattern_zalessky.findall(zalessky_list)
matches = [value.split(" ")[0] + " " + value.split(" ")[1].lower().capitalize() + " " + value.split(" ")[-1].lower().capitalize() for value in matches]
zalessky_matches = matches

zalessky_matches_list = []
counter = 0
for name in zalessky_matches:
    if name in list_names_4041:
        counter +=1
        zalessky_matches_list.append(name)

print(f"Matches in Zalessky: {counter}")

Matches in Zalessky: 98


### Combine all matches

In [ ]:
all_matches = set(list(lurie_matches + komandarmy_matches_list + komcory_matches + komdivy_matches + 
                       repressions_matches + tarkhova_matches_list + petrov_skorkin_matches_list + zalessky_matches_list))
len(all_matches)

In [ ]:
1280 - 783

In [ ]:
counter = 0
for name in list_names_4041:
    if not name in all_matches:
        counter +=1
        print(name)
print(counter)

In [ ]:
df = pd.read_csv("generals_admirals_dataset.csv")
df